# Model Training

In [59]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import importlib
import os
import sys
# module = os.path.abspath('/home/learner/wesad_experiments/src/main')
module = os.path.abspath('C:/github/stress-affect-wesad/src/main')
if module not in sys.path:
    sys.path.append(module)
from DataManager import DataManager
from Demo import Demo

Here we will go ahead and do the work to prepare the data for NN creation and actually create the neural network.

In [60]:
manager = DataManager()

In [61]:

manager.load_all()


Loading data for S2
Loading data for S3
Loading data for S4
Loading data for S5
Loading data for S6
Loading data for S7
Loading data for S8
Loading data for S9
Loading data for S10
Loading data for S11


In [62]:
manager.compute_features();
manager.compute_features_stress();

Computing features..
	subject: 2
	subject: 3
	subject: 4
	subject: 5
	subject: 6
	subject: 7
	subject: 8
	subject: 9
	subject: 10
	subject: 11
conputing features..
	subject: 2
	subject: 3
	subject: 4
	subject: 5
	subject: 6
	subject: 7
	subject: 8
	subject: 9
	subject: 10
	subject: 11


In [63]:
print("We have", len(manager.SUBJECTS), " subjects")

for feature in manager.FEATURES.keys():
    print("there are ", len(manager.FEATURES[feature]), " values for ", feature)
#     print(manager.FEATURES[feature][3277:3288])

We have 10  subjects
there are  177844  values for  a_mean
there are  177844  values for  a_std
there are  177844  values for  a_maxx
there are  177844  values for  a_maxy
there are  177844  values for  a_maxz
there are  177844  values for  e_max
there are  177844  values for  e_min
there are  177844  values for  e_mean
there are  177844  values for  e_range
there are  177844  values for  e_std
there are  177844  values for  t_max
there are  177844  values for  t_min
there are  177844  values for  t_mean
there are  177844  values for  t_range
there are  177844  values for  t_std


We need to reshape the data such that there are [samples, features].

In [64]:
# Lets go ahead and reshape this data such that we have
# samples, features = [N, 15]

X1 = []
X2 = []
for i in range(0,  len(manager.FEATURES['a_mean'])):
    X1.append([manager.FEATURES['a_mean'][i], manager.FEATURES['a_std'][i], manager.FEATURES['a_maxx'][i], manager.FEATURES['a_maxy'][i], manager.FEATURES['a_maxz'][i],\
                  manager.FEATURES['e_max'][i], manager.FEATURES['e_min'][i], manager.FEATURES['e_mean'][i], manager.FEATURES['e_range'][i], manager.FEATURES['e_std'][i],\
                  manager.FEATURES['t_max'][i], manager.FEATURES['t_min'][i], manager.FEATURES['t_mean'][i], manager.FEATURES['t_range'][i], manager.FEATURES['t_std'][i]])
print(np.shape(X1))

for i in range(0,  len(manager.STRESS_FEATURES['a_mean'])):
    X2.append([manager.STRESS_FEATURES['a_mean'][i], manager.STRESS_FEATURES['a_std'][i], manager.STRESS_FEATURES['a_maxx'][i], manager.STRESS_FEATURES['a_maxy'][i], manager.STRESS_FEATURES['a_maxz'][i],\
                  manager.STRESS_FEATURES['e_max'][i], manager.STRESS_FEATURES['e_min'][i], manager.STRESS_FEATURES['e_mean'][i], manager.STRESS_FEATURES['e_range'][i], manager.STRESS_FEATURES['e_std'][i],\
                  manager.STRESS_FEATURES['t_max'][i], manager.STRESS_FEATURES['t_min'][i], manager.STRESS_FEATURES['t_mean'][i], manager.STRESS_FEATURES['t_range'][i], manager.STRESS_FEATURES['t_std'][i]] )
print(np.shape(X2))

(177844, 15)
(95120, 15)


In [65]:
# print(manager.FEATURES['a_mean'][100:110])

# print(manager.FEATURES['a_std'][400:410])

# print(manager.FEATURES['a_maxx'][0:10])

In [66]:
import numpy as np
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import tensorflow as tf
import keras
from keras.preprocessing import sequence

from keras.models import Sequential

from keras.layers import Dense, Embedding, Flatten

from keras.layers import LSTM

In [67]:
# from keras import backend as K
# K.tensorflow_backend._get_available_gpus()

# session_conf = tf.ConfigProto(intra_op_parallelism_threads=8, inter_op_parallelism_threads=8, )

# tf.set_random_seed(1)

# session = tf.Session(graph=tf.get_default_graph(), config=session_conf)
# keras.backend.set_session(session)

Initialize two arrays for the targets.

In [68]:
# x1 is the baseline data
# x2 is the stress data

y1 = [0] * len(X1)
y2 = [1] * len(X2)

In [69]:
print(len(y1))
print(len(y2))

177844
95120


Now we need to concat the data between baseline and stress so that we can split it into training and test sets

In [70]:
X = np.concatenate((X1, X2), axis=0)
print(np.shape(X))

y = np.concatenate((y1,y2), axis=0)
print(np.shape(y))

(272964, 15)
(272964,)


Split the data up in train and test sets


In [71]:
from sklearn.model_selection import train_test_split

# since we don't have the same number of samples for x and y, 
# we will drop off some x values for creating the sample
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.25, random_state=42)

In [72]:
print(np.shape(X_train))
print(np.shape(X_test))
print(np.shape(y_train))
print(np.shape(y_test))

(204723, 15)
(68241, 15)
(204723,)
(68241,)


Need to scale the data and hope we get better results.
1. fit the scaler to the training data (fit_transform)
2. transform training data with the scaler
3. fit the model with transformed data
4. transform test data with the scaler
5. predict using model and output of (4)

It will be better to have our data scaled between 0 and 1, so lets go ahead and create a function to 
normalize the data that way now.

In [73]:
def normalize(data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    return scaler.fit_transform(data)
X_train = normalize(X_train)
X_test = normalize(X_test)

Should be
input_shape=(number of sequences=?, time_steps=None, features=15)
target=(number of sequences, time_steps, targets)

We will use time_steps = 1 since our data has implicit timesteps from the feature calculations.

In [74]:
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [75]:
print(np.shape(X_train))
print(np.shape(X_test))
print(np.shape(y_train))
print(np.shape(y_test))

(204723, 1, 15)
(68241, 1, 15)
(204723,)
(68241,)


In [76]:
num_neurons = 15
num_features = 15

In [77]:
print('Building the LSTM network...')

model = Sequential()
model.add(LSTM(num_neurons, input_shape=(1, num_features), return_sequences=True))
# model.add(LSTM(16, input_shape=(1, 15), dropout=0.35, recurrent_dropout=0.35, return_sequences=True))
# Note:
# Need to do return_sequences=False for the layer *before* dense
# https://stackoverflow.com/questions/51763983/error-when-checking-target-expected-dense-1-to-have-3-dimensions-but-got-array
model.add(LSTM(num_neurons, input_shape=(1, num_features), return_sequences=False))
model.add(Dense(1, activation='sigmoid'))


Building the LSTM network...


In [78]:
model.summary()

print("inputs: " , model.input_shape)
print("outputs: ", model.output_shape)
print("actual inputs: ", np.shape(X_train))
print("actual outputs: ", np.shape(y_train))

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 1, 15)             1860      
                                                                 
 lstm_7 (LSTM)               (None, 15)                1860      
                                                                 
 dense_3 (Dense)             (None, 1)                 16        
                                                                 
Total params: 3,736
Trainable params: 3,736
Non-trainable params: 0
_________________________________________________________________
inputs:  (None, 1, 15)
outputs:  (None, 1)
actual inputs:  (204723, 1, 15)
actual outputs:  (204723,)


In [79]:
from keras.optimizers import SGD
opt = SGD(lr=0.05)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

c:\Users\dollj\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [80]:
print('Training LSTM...')

batch_size = 2 # I think I want to use batch_size = 1

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=1,
          validation_data=(X_test, y_test))

score, acc = model.evaluate(X_test, y_test, batch_size=batch_size)

Training LSTM...
34121/34121 [==============================] - 29s 848us/step - loss: 0.0856 - accuracy: 0.9696


In [81]:
print('score:', score)
print('accuracy:', acc)

score: 0.0855560228228569
accuracy: 0.9695930480957031


In [82]:
# Let's save the model to disc
json = model.to_json()
with open("model.json", "w") as file:
    file.write(json)
model.save_weights("model.h5")
print("Saved model to disc")

Saved model to disc


In [83]:
from keras.models import model_from_json
# Load the model of interest
json_file = open('model.json', 'r')
json = json_file.read()
json_file.close()
model_from_disc = model_from_json(json)
model_from_disc.load_weights("model.h5")

In [84]:
opt = SGD(lr=0.05)
model_from_disc.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

score, acc = model_from_disc.evaluate(X_test , y_test, batch_size=batch_size)
print('score:', score)
print('accuracy:', acc)

34121/34121 [==============================] - 32s 909us/step - loss: 0.0856 - accuracy: 0.9696
score: 0.0855560228228569
accuracy: 0.9695930480957031


In [85]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
y_pred = model_from_disc.predict(X_test, batch_size=2, verbose=1)

34121/34121 [==============================] - 28s 805us/step


In [86]:
print(y_pred)

[[2.0262085e-05]
 [3.2748954e-05]
 [2.0046016e-05]
 ...
 [7.7930144e-03]
 [2.0588031e-05]
 [9.0125281e-01]]


In [87]:
y_pred[y_pred>0.5] = 1 
y_pred[y_pred<=0.5] = 0 

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98     44258
           1       0.99      0.92      0.96     23983

    accuracy                           0.97     68241
   macro avg       0.98      0.96      0.97     68241
weighted avg       0.97      0.97      0.97     68241

[[44067   191]
 [ 1884 22099]]


Performance using the LSTM based network architecture with one hidden layer
has performed with an accuracy of ~ 97.4% on the validation set using 5 epochs.

With just one epoch, the model has results between ~80% and ~92% for accuracy
on the validation data. Each epoch of training takes approximately 70 seconds
without GPU acceleration. At 5 epochs, the model outperforms the WESAD quoated
results for both accuracy and F1 using less modalities and less features.